<center style="font-size: 26px"> <b>Проверка уровня знаний Python</b></center>

In [1]:
# обеспечиваем совместимость с Python 2 и 3
from __future__ import (absolute_import, division, print_function, unicode_literals)

# отключаем предупреждения дистрибутива Anaconda
import warnings
warnings.simplefilter('ignore')

**Описание датасета:**   
[Датасет вакансий с платформы HH.ru на Kaggle.com](https://www.kaggle.com/datasets/pavfedotov/heaadhunter-vacancies?resource=download&select=df2021-08-03.csv) 

Нужный файл - df2021-08-03.csv.

- *vacancy* - наименование вакансии
- *url* - ссылка на вакансию
- *created* - дата и время создания
- *has_test* - наличие тестового задания в вакансии
- *salary_from* - нижняя граница значения заработной платы
- *salary_to* - верхняя граница значения заработной платы
- *currency* - валюта заработной платы
- *experience* - требуемый опыт
- *schedule* - тип рабочего графика
- *skills* - требуемые навыки
- *employer* - наименование работодателя
- *area* - наименование города
- *description* - описание вакансии

#### Загрузите датасет

In [85]:

import pandas as pd

df = pd.read_csv('df2021-08-03.csv')

In [86]:
df['created'] = pd.to_datetime(df['created'], format='%Y-%m-%dT%H:%M:%S%z')
df['created'] = df['created'].dt.strftime('%Y-%m-%d %H:%M:%S')

In [87]:
df[:2]

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description
0,Backend/Full-stack developer (python),https://hh.ru/applicant/vacancy_response?vacan...,2021-08-01 13:02:48,False,120000.0,NaN,RUR,От 1 года до 3 лет,Гибкий график,Python;PostgreSQL;Linux;Flask;,ATI.SU,Санкт-Петербург,Привет! Мы ATI.SU ― крупнейшая в России компан...
1,Бэкенд-разработчик (Python) / Middle Python / ...,https://hh.ru/applicant/vacancy_response?vacan...,2021-07-09 08:13:01,False,150000.0,220000.0,RUR,От 3 до 6 лет,Удаленная работа,Python;Git;MongoDB;Redis;Design Patterns;Flask;,"ЮТэйр, Авиакомпания",Киров (Кировская область),Utair - российская авиакомпания. Мы летаем по ...


In [88]:
df['currency'].unique()

array(['RUR', nan, 'EUR', 'USD'], dtype=object)

### Практические задания:

#### 1. Изучите методы для сбора данных курсов валют в [API Центрального банка Российской Федерации](https://www.cbr.ru/development/SXML/) . Используя запросы к API, приведите значения заработной платы к рублям одним из вариантов: 1) простой вариант - на текущую дату  2) вариант посложнее - на дату создания (created).

In [89]:

import requests
import numpy as np
from datetime import datetime

# Функция для получения курса валют на дату
def get_currency_rate(date):
    url = f'http://www.cbr.ru/scripts/XML_daily.asp?date_req={date}'
    response = requests.get(url)
    response.encoding = 'utf-8'
    content = response.text
    rate = content.split('<Valute ID="R01235">')[1].split('<Value>')[1].split('</Value>')[0]
    return float(rate.replace(',', '.'))

# Функция для преобразования зарплаты в рубли на основе курса валют
def convert_salary_to_rubles(salary_from, salary_to, currency, created):
    if currency == 'RUR':
        return np.nanmean([salary_from, salary_to])
    elif pd.isna(currency):
        return None
    else:
        try:
            rate_date = datetime.strptime(created, '%Y-%m-%d %H:%M:%S').strftime('%d/%m/%Y')
            rate = get_currency_rate(rate_date)
        except:
            return None
        if currency == 'USD':
            return round(np.nanmean([salary_from, salary_to]) * rate, 2)
        elif currency == 'EUR':
            return round(np.nanmean([salary_from, salary_to]) * rate, 2)    
        else:
            return None

# Создание нового столбца с зарплатой в рублях
df['mean_salary_rub'] = df.apply(lambda x: convert_salary_to_rubles(x['salary_from'], x['salary_to'], x['currency'], x['created']), axis=1)


In [90]:
df[df['currency'] == 'USD'][:1]

,vacancy,url,created,has_test,salary_from,salary_to,currency,experience,schedule,skills,employer,area,description,mean_salary_rub
142,"Python Developer (Люксембург, удаленно)",https://hh.ru/applicant/vacancy_response?vacan...,2021-07-30 13:09:00,False,NaN,3500.0,USD,От 3 до 6 лет,Удаленная работа,Python;,Кадровое агентство Алексея Сухорукова,Новосибирск,"IT-компания, которая предоставляет своим клиен...",256166.4


#### 2. Найдите вакансию с самой высокой заработной платой, где подойдет 4 года опыта работы и не нужно проходить тестовое задание.

In [91]:
import re

# Приведение значений столбца experience к строковому типу
df['experience'] = df['experience'].astype(str)

# Создание столбца experience_min
df['experience_min'] = df['experience'].apply(lambda x: re.findall('\d+', x)[0] if re.findall('\d+', x) else None)

# Приведение значений столбца experience_min к числовому типу
df['experience_min'] = pd.to_numeric(df['experience_min'], errors='coerce')

# Выборка вакансий с опытом работы не более 4 лет
df_filtered = df[(df['experience_min'] <= 4) & (df['has_test'] == False)]

# Удаление строк с пропущенными значениями зарплаты
df_filtered.dropna(subset=['experience_min'], inplace=True)

# найти вакансию с самой высокой зарплатой
highest_salary = df_filtered.loc[df_filtered['mean_salary_rub'].idxmax()]

# вывести информацию о вакансии с самой высокой зарплатой
print("Вакансия с наибольшей средней зарплатой:")
print(highest_salary['vacancy'])
print(highest_salary['url'])
print(highest_salary['mean_salary_rub'], "руб.")


Вакансия с наибольшей средней зарплатой:
Python разработчик (middle) (backend)
https://hh.ru/applicant/vacancy_response?vacancyId=43941903
500000.0 руб.


#### 3. Сделайте рейтинг (топ-30) навыков по всем вакансиям.

In [139]:

# import plotly.express as px
# import plotly.subplots as sp
# import plotly.graph_objs as go

# # создание списка уникальных навыков
# skills_set = set()
# for skills in df['skills']:
#     if isinstance(skills, str):
#         skills_list = skills.split(';')
#         skills_set.update(skills_list)

# # создание новых столбцов для каждого навыка
# for skill in skills_set:
#     df[skill] = df['skills'].apply(lambda x: int(skill in x.split(';')) if isinstance(x, str) else 0)

# # Считаем относительную частоту навыков
# for skill in skills_set:
#     df[skill + '_freq'] = df.apply(lambda row: row[skill] / sum(x for x in row[skills_set] if x) if row[skill] != 0 else 0, axis=1)

# # Считаем произведение относительной частоты на среднюю зарплату
# for skill in skills_set:
#     df[skill + 'a_salary'] = df[skill] * df['mean_salary_rub']

# # Считаем произведение относительной частоты на среднюю зарплату
# for skill in skills_set:
#     df[skill + 'f_salary'] = df[skill + '_freq'] * df['mean_salary_rub']

# выбираем топ-30 наиболее употребляемых навыков по 4 различным стратегиям
skills_count_top30 = df.iloc[:, 16:850].sum().sort_values(ascending=False)[:30].reset_index().rename(columns={0: 'count'})
skills_freq_top30 = df.filter(regex='_freq$').mean().sort_values(ascending=False)[1:31].reset_index().rename(columns={0: 'freq'})
skills_a_salary_top30 = df.filter(regex='a_salary$').mean().sort_values(ascending=False)[1:31].reset_index().rename(columns={0: 'a_salary'})
skills_f_salary_top30 = df.filter(regex='f_salary$').mean().sort_values(ascending=False)[1:31].reset_index().rename(columns={0: 'f_salary'})
skills_freq_top30['index'] = skills_freq_top30['index'].str.replace('_freq', '')
skills_a_salary_top30['index'] = skills_a_salary_top30['index'].str.replace('a_salary', '')
skills_f_salary_top30['index'] = skills_f_salary_top30['index'].str.replace('f_salary', '')

# Создаем новый столбец rank
skills_count_top30['rank'] = list(reversed(range(1, 31)))
skills_freq_top30['rank'] = list(reversed(range(1, 31)))
skills_a_salary_top30['rank'] = list(reversed(range(1, 31)))
skills_f_salary_top30['rank'] = list(reversed(range(1, 31)))

skills_count_top30 = skills_count_top30.set_index('index')
skills_freq_top30 = skills_freq_top30.set_index('index')
skills_a_salary_top30 = skills_a_salary_top30.set_index('index')
skills_f_salary_top30 = skills_f_salary_top30.set_index('index')

# Объединяем все наборы данных в один
top_skills = pd.concat([skills_count_top30, skills_freq_top30, skills_a_salary_top30, skills_f_salary_top30], axis=1, join='outer')

# Суммируем баллы по всем 4 рейтингам
top_skills['total_score'] = top_skills['rank'].sum(axis=1)

# # Выводим топ 30 на график
# fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Топ 30 наиболее упоминаемых навыков", "Топ 30 наиболее востребованных навыков", "Топ 30 высокооплачиваемых навыков", "Топ 30 высокооплачиваемых навыков на вакансию"), shared_xaxes=True)

# # Строим график
# fig.add_trace(
#     go.Bar(x=skills_count_top30.index, y=skills_count_top30.values, name='Топ 30 наиболее упоминаемых навыков',
#            marker_color=get_colors_list(skills_count_top30, colors)), row=1, col=1)

# fig.add_trace(
#     go.Bar(x=skills_freq_top30.index, y=skills_freq_top30.values, name='Топ 30 наиболее востребованных навыков',
#            marker_color=get_colors_list(skills_freq_top30, colors)), row=1, col=2)

# fig.add_trace(
#     go.Bar(x=skills_a_salary_top30.index, y=skills_a_salary_top30.values, name='Топ 30 высокооплачиваемых навыков',
#            marker_color=get_colors_list(skills_a_salary_top30, colors)),row=2, col=1)

# fig.add_trace(
#     go.Bar(x=skills_f_salary_top30.index, y=skills_f_salary_top30.values, name='Топ 30 высокооплачиваемых навыков',
#            marker_color=get_colors_list(skills_f_salary_top30, colors)),row=2, col=1)

# # Настройки внешнего вида графика
# fig.update_layout(height=900, width=1200, title_text="Анализ навыков вакансии Data Scientist", font=dict(size=16), showlegend=False)

# # Настройки для графиков
# fig.update_xaxes(tickangle=45, tickfont=dict(size=14))
# fig.update_yaxes(tickfont=dict(size=14))

# fig.show()


In [148]:
top_30_skills = top_skills.sort_values('total_score', ascending=False)[:30]
top_30_skills

,count,rank,freq,rank,a_salary,rank,f_salary,rank,total_score
index,,,,,,,,,
Python,1442.0,30.0,0.150592,30.0,134830.227651,30.0,22842.249041,30.0,120.0
Git,586.0,29.0,0.050512,29.0,63351.774796,29.0,8821.456031,29.0,116.0
PostgreSQL,499.0,27.0,0.044653,27.0,57062.739347,28.0,8384.920698,28.0,110.0
SQL,501.0,28.0,0.045298,28.0,41507.884910,25.0,5634.315108,26.0,107.0
Django Framework,467.0,26.0,0.044081,26.0,53509.486166,27.0,8251.678670,27.0,106.0
Linux,463.0,25.0,0.041882,25.0,42719.457423,26.0,5574.225736,25.0,101.0
Docker,172.0,23.0,0.013353,23.0,18824.830343,22.0,2081.950688,22.0,90.0
Flask,177.0,24.0,0.014433,24.0,15326.264274,20.0,1994.465228,20.0,88.0
MySQL,128.0,20.0,0.010410,20.0,19395.000196,23.0,2503.791745,23.0,86.0


In [152]:
def get_categories(data):
    """
    Разбивает индекс переданного DataFrame на 3 категории и возвращает список из этих категорий
    """
    categories = [
        data.index[0:6],
        data.index[6:17],
        data.index[17:30]
    ]
    return categories

def get_colors_list(data, colors):
    """
    Возвращает список цветов для каждой категории индекса переданного DataFrame
    """
    colors_list = np.select(
        [data.index.isin(category) for category in get_categories(data)],
        colors
    )
    return colors_list

# Список цветов для категорий
colors = ['> 100', 'от 50 до 100', '< 50']

# Выводим топ 30 на график
# fig = sp.make_subplots(rows=2, cols=2, subplot_titles=("Трп 30 часто встречающихся навыков", "Топ 30 наиболее востребованных навыков", "Топ 30 высокооплачиваемых навыков", "Топ 30 высокооплачиваемых навыков на вакансию"))


# Строим график
fig = px.bar(top_30_skills, x=top_30_skills.index, y=top_30_skills['total_score'], 
             labels={'x':'Навык', 'y':'Количество упоминаний'}, 
             title='Топ-30 наиболее восстребованных навыков',
             color=get_colors_list(top_30_skills, colors))
             
# Изменяем название легенды
fig.update_layout(legend_title_text='Категории')


fig.show()

In [153]:

fig = px.pie(top_skills, values='total_score', names=top_skills.index)
fig.show()


#### 4. Cоставьте топ-10 городов с наибольшей средней заработной платой по вакансиям тестировщика (QA).

In [ ]:
# ВАШ КОД ЗДЕСЬ


#### 5. Покажите динамику количества вакансии по месяцам даты создания.

In [ ]:
# ВАШ КОД ЗДЕСЬ


#### 6. Напишите функцию для поиска вакансий по переданным в нее значениям заработной платы, наличия тестового задания, наличия заданного текста в наименовании вакансии, наличия заданного навыка. В результате выведите количество найденных вакансий, среднюю заработную плату по найденным вакансиям и список ссылок найденных вакансий. 

In [ ]:
# ВАШ КОД ЗДЕСЬ


#### *Необязательно*. Усложненный вариант - сделать атрибуты функции опциональными, чтобы иметь возможность фильтрации вакансий по любой комбинации условий.

In [ ]:
# ВАШ КОД ЗДЕСЬ
